In [1]:
from platform import python_version  
print(python_version())

3.13.1


In [2]:
import sys, os
sys.path.append('../dependencies/')
import mfsetup
import flopy as fp
import flopy.utils as fu
from flopy.utils.gridintersect import GridIntersect
import shapefile as sf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from shapely.geometry import Point, LineString
import geopandas as gp
datapath = Path('../data/sgn/')# stdir = os.getcwd()

In [ ]:
m = mfsetup.MF6model.setup_from_yaml('sgn_config.yml')  # ___________________________________ sgn_mfsetup ___ Folder

In [ ]:
m.write_input()

In [5]:
ix = GridIntersect(m.modelgrid)

In [6]:
boundary = gp.read_file('../../data/sgn/shp/Model_domain.shp')
boundary_line =  LineString([Point(i) for i in boundary.iloc[0].geometry.exterior.coords])    # boundary_line
ix_boundary = ix.intersect(boundary_line.buffer(25, resolution=4))
ix_df = pd.DataFrame.from_records(ix_boundary)   # ix_df.head()

In [7]:
sep19_head = fu.Raster.load('../../data/sgn/csv/heads_sep2019.asc')  # OBS heads to make the GHB data
# plt.imshow(sep19_head.get_array(1))  # plt.colorbar()

In [8]:
# resample these heads to the model
head = sep19_head.resample_to_grid(m.modelgrid,band=sep19_head.bands[0], method="nearest")

In [9]:
ghb_spd = []                         # make the GHB stress period data
for icpl in ix_boundary.cellids:
    cond = 100.0
    for layer in [0,2]:
        # note - structured grid so *icpl unpacks row/col
        ghb_spd.append([(layer, *icpl), head[icpl], cond, f'GHB{layer+1}'])    # ghb_spd

In [10]:
ghb = fp.mf6.ModflowGwfghb(model=m, stress_period_data=ghb_spd, boundnames=True) 

In [34]:
maw_wells = pd.read_csv('../../data/sgn/wells_zero.csv', index_col = 0)
# maw_wells.head()
maw_wells['cellid']=[ix.intersect(Point(x,y))[0][0] for x,y in zip(maw_wells.x,maw_wells.y)]
# maw_wells.head()    # reset the index for MAW wells to be sequential only among the remaining wells
maw_wells['top'] = [m.dis.top.array[r,c] for r,c in maw_wells.cellid]               # multi aquifer wells - MAW
maw_wells['bot'] = [m.dis.botm.array[-1,r,c] for r,c in maw_wells.cellid]
maw_wells['strt'] = [m.ic.strt.array[0,r,c] for r,c in maw_wells.cellid]
maw_radius = 0.1         # packagedata : [wellno, radius, bottom, strt, condeqn, ngwfnodes, aux,  boundname] where:
maw_packagedata = []
for wellno,cwell in maw_wells.iterrows(): maw_packagedata.append([wellno, maw_radius, cwell.bot, cwell.strt, 'THIEM', 3])  # maw_packagedata
maw_connections = []   # connectiondata : [wellno, icon, cellid, scrn_top, scrn_bot, hk_skin, radius_skin] where
for wellno,cwell in maw_wells.iterrows():
    for clay in range(3):
        maw_connections.append([wellno, clay, clay, cwell.cellid[0], cwell.cellid[1], cwell.top, cwell.bot, -999, -999])  # maw_connections
maw_spd = [[i, 'rate', 0] for i in range(len(maw_wells))]

In [35]:
include_MAW = True

In [38]:
if include_MAW:
    maw = fp.mf6.ModflowGwfmaw(model=m,no_well_storage=True,nmawwells=len(maw_wells),
                               packagedata=maw_packagedata,connectiondata=maw_connections,perioddata=maw_spd,)
    obs_file = f"{m.name}.maw.obs"         # set up some observations
    csv_file = f"{obs_file}.csv"
    head_obs =  [['head','head', i+1] for i in range(len(maw_wells))]
    Qobs = []
    for i in range(len(maw_wells)):
        Qobs.extend([[f'Q.{i}_lay{j}', 'maw', i+1, j+1 ] for j in range(3)])
    obs_dict = {csv_file: head_obs + Qobs}
    maw.obs.initialize(filename = obs_file, digits  = 8, print_input=True, continuous=obs_dict)

MFDataException: An error occurred in data element "packagedata" model "sgn_50" package "maw_3". The error occurred while setting data in the "__init__" method.
Additional Information:
(1) Error occurred while adding dataset "packagedata" to block "packagedata"

In [ ]:
m.write_input()

In [21]:
m.name

'sgn_50'

In [22]:
m.dis.export('testgrid.shp')

In [23]:
ims = [i.rstrip() for i in open('../../models/sgn_mfsetup/sgn_50_sim.ims').readlines()]  # ims

In [24]:
with open('../../models/sgn_mfsetup/sgn_50_sim.ims', 'w') as ofp:
    [ofp.write("{}\n".format(line.replace('END options', '  NO_PTC ALL\nEND options'))) for line in ims]

In [25]:
m.simulation.run_simulation()

FloPy is using the following executable to run the model: mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                   VERSION 6.3.0 release candidate 02/06/2022
                               ***DEVELOP MODE***

   MODFLOW 6 compiled Feb 06 2022 02:35:51 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.5.0
                             Build 20211109_000000

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided on the 
condition that neither the USGS nor the U.S. Gove

(False, [])

In [ ]:
top = m.dis.top.array

In [ ]:
plt.imshow(top)

In [ ]:
h = fu.binaryfile.HeadFile('../../models/sgn_mfsetup/sgn_50.hds').get_alldata()

In [ ]:
h.shape

In [ ]:
plt.imshow(np.squeeze(h)[0][0])
plt.colorbar()

In [ ]:
plt.imshow(np.squeeze(h)[1][0])
plt.colorbar()

### make sure the barrier well shows up in the second stress period

In [ ]:
plt.imshow(np.squeeze(h)[1][0]-np.squeeze(h)[0][0])
plt.colorbar()

### check flooding

In [ ]:
flooded = top-np.squeeze(h)[0][0]
plt.imshow(flooded)
plt.colorbar()

In [ ]:
flooded[flooded>0] = np.nan
plt.imshow(flooded)
plt.colorbar()

In [ ]:
df1,df2 = fu.Mf6ListBudget('../../models/sgn_mfsetup/sgn_50.list').get_dataframes(
    start_datetime='2020-12-31')

In [ ]:
df1

In [ ]:
df2

## check out the MAW obs

In [ ]:
if include_MAW:
    maw_df = pd.read_csv('../../models/sgn_mfsetup/sgn_50.maw.obs.csv', index_col=0)

In [ ]:
maw_df